In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import gurobipy_pandas as gppd

# set intercative mode
gppd.set_interactive()

In [2]:
# importing data
hubs = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_hubs.csv")
network = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_network.csv")
plants = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_plants.csv")
roads = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_roads.csv")
suppliers = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_suppliers.csv")
railroads = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_railroads.csv")

In [3]:
hubs.set_index('hub', inplace=True)
hubs.head()

,index,latitude,longitude,invest,capacity
hub,,,,,
17201,1,33.64844,-95.56841,3476219,300000
17218,2,33.64124,-96.60666,3476219,300000
17359,3,29.31990,-96.10283,3476219,300000
17372,4,30.75623,-98.67770,3476219,300000
17395,5,30.84636,-96.98711,3476219,300000


In [4]:
network.head()

,counties,hubs,plants,techs,demand
0,254,33,167,1,1476310602


In [5]:
plants.set_index('plant', inplace=True)
plants.head()

,index,latitude,longitude,tech,invest,capacity,yield
plant,,,,,,,
541,1,30.220353,-97.740436,1,130956797,152063705,232
542,2,29.676724,-98.635234,1,130956797,152063705,232
543,3,33.760315,-96.559867,1,130956797,152063705,232
544,4,29.531920,-98.286901,1,130956797,152063705,232
545,5,29.688568,-98.562424,1,130956797,152063705,232


In [6]:
roads.set_index(['county','hub'], inplace=True)
roads.head()

index  distance       cost
county hub                              
48001  17201      1  219.6090  40.580691
       17218      2  264.7658  47.661277
       17359      3  370.6750  64.267840
       17372      4  420.0082  72.003286
       17395      5  199.2915  37.394907

In [7]:
(roads['cost']/roads['distance']).mean()

0.1783104491604194

In [8]:
# cleaning dataset by removing NA values
suppliers.dropna(inplace=True)
suppliers.county = suppliers.county.astype(int)
suppliers.set_index('county',inplace=True)
suppliers.head()

,index,supply
county,,
48001,1.0,13131.971710
48003,2.0,1177.351950
48005,3.0,3854.618542
48007,4.0,308.182629
48009,5.0,19802.136510


In [9]:
suppliers['supply'].sum()*232/network['demand']

0    0.479834
Name: demand, dtype: float64

In [10]:
railroads.set_index(['hub','plant'], inplace=True)
railroads.head()

index     distance       cost  loading  capacity
hub   plant                                                  
17201 541        1   922.023251  17.095711  3066792    338000
      542        2  1160.842967  20.923752  3066792    338000
      543        3   186.924258   5.312809  3066792    338000
      544        4  1074.699073  19.542951  3066792    338000
      545        5  1160.842967  20.923752  3066792    338000

In [11]:
(railroads['cost']/railroads['distance']).mean()

0.02091824870583491

In [12]:
# create gurobi model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-17


In [13]:
### Variables
yhub = gppd.add_vars(m, hubs, name='yhub', vtype=GRB.BINARY, lb=0)
yplant = gppd.add_vars(m, plants, name='yplant', vtype=GRB.BINARY, lb=0)
ytruck = gppd.add_vars(m, roads, name='ytruck', vtype=GRB.BINARY, lb=0)
ytrain = gppd.add_vars(m, railroads, name='ytrain', vtype=GRB.BINARY, lb=0)

c_to_h = gppd.add_vars(m, roads, name='c_to_h', vtype=GRB.CONTINUOUS, lb=0) # Flow from county to hub
h_to_p = gppd.add_vars(m, railroads, name='h_to_p', vtype=GRB.CONTINUOUS, lb=0) # Flow from hub to plant

x_out = m.addVar(vtype=GRB.CONTINUOUS, name='outside_supply', lb=0) #biofuel supply from outside


In [14]:
# y_third_party = gppd.add_vars(m, name='y_third_party', vtype=GRB.BINARY)                                    )

In [15]:
### parameters
"""
demand = network['demand]
yield = plants['yield']
cplant = plants['invest']
chub = hubs['invest']
"""
M = 500000
# fuel costfrom external source
c_out = 1.5

In [16]:
### Objective
# divide by 1000 to ease up computation
m.setObjective(((plants['invest']*0.001 * yplant).sum() + (hubs['invest']*0.001 * yhub).sum() + (c_to_h * roads['cost'] * 0.001).sum() \
               + (ytrain * railroads['loading'] * 0.001).sum() + (h_to_p * railroads['cost'] * 0.001).sum() + c_out*x_out*0.001), 
               GRB.MINIMIZE)

In [17]:
### Highest possible demand that can be met by the network
#m.setObjective((yplant * plants['yield'] * h_to_p.groupby('plant').sum()).sum(), GRB.MAXIMIZE)

In [18]:
### Constraints
# if only unmet demands fulfilled by third-party
# third_party_supply = gppd.add_constrs(m,
#                                     x_out,
#                                     GRB.LESS_EQUAL,
#                                     network['demand'] - (h_to_p.groupby('plant').sum()).sum(),
#                                     name='third_party_supply'
# )

In [19]:
# flow conservation through hub
flow_hub = gppd.add_constrs(m,
                            c_to_h.groupby('hub').sum(),
                            GRB.EQUAL,
                            h_to_p.groupby('hub').sum(),
                            name='hub_flow'
                            )

In [20]:
# hub capacity
cap_hub = gppd.add_constrs(m,
                           c_to_h.groupby('hub').sum(),
                           GRB.LESS_EQUAL,
                           hubs['capacity']*yhub,
                           name='hub_capacity'
                           )

In [21]:
# train capacity
cap_train = gppd.add_constrs(m,
                           h_to_p,
                           GRB.LESS_EQUAL,
                           railroads['capacity']*ytrain,
                           name='train_capacity'
                           )

In [22]:
# supplier (county) capacity
cap_county = gppd.add_constrs(m,
                           (c_to_h).groupby('county').sum(),
                           GRB.LESS_EQUAL,
                           suppliers['supply'],
                           name='suppliers_supply'
                           )

In [23]:
# delivery by truck
truck_delivery = gppd.add_constrs(m,
                           c_to_h,
                           GRB.LESS_EQUAL,
                           M*ytruck,
                           name='truck_binary'
                           )

# delivery by train
train_delivery = gppd.add_constrs(m,
                           h_to_p,
                           GRB.LESS_EQUAL,
                           M*ytrain,
                           name='train_binary'
                           )

In [24]:
# plant capacity
cap_plant = gppd.add_constrs(m,
                            h_to_p.groupby('plant').sum(),
                            GRB.LESS_EQUAL,
                            (plants['capacity']/plants['yield'])*yplant,
                            name='plant_capacity'
                            )

In [25]:
# demand
demand_constraint = gppd.add_constrs(m,
                                     (x_out/232) + (h_to_p.groupby('plant').sum()).sum(),
                                     GRB.GREATER_EQUAL,
                                     network['demand']/232,
                                     name='network_demand'
                                     )

In [26]:
# write the model
m.write("supply_chain_biofuel_with_outside_supply.lp")

In [27]:
m.params.LogToConsole = 1
m.params.MIPgap = 0.02 ###important
m.params.ScaleFlag = 1
m.params.MIPFocus = 3
m.params.BranchDir = -1
m.params.Cuts = 3
m.params.NumericFocus = 1

m.update()
m.optimize()

Set parameter MIPGap to value 0.02
Set parameter ScaleFlag to value 1
Set parameter MIPFocus to value 3
Set parameter BranchDir to value -1
Set parameter Cuts to value 3
Set parameter NumericFocus to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 19892 rows, 27987 columns and 80688 nonzeros
Model fingerprint: 0x8d3c7a91
Variable types: 13894 continuous, 14093 integer (14093 binary)
Coefficient statistics:
  Matrix range     [4e-03, 7e+05]
  Objective range  [2e-03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 6e+06]
Found heuristic solution: objective 2214465.9030
Presolve removed 13899 rows and 8580 columns
Presolve time: 0.07s
Presolved: 5993 rows, 19407 columns, 52308 nonzeros
Variable types: 13696 continuous, 5711 integer (5711 binary)
Root relaxation presolve 

In [ ]:
# m.computeIIS()

In [28]:
hub_to_plant = [m.getVarByName(f"h_to_p[{i},{j}]").X for i in hubs.index for j in plants.index]
county_to_hub = [m.getVarByName(f"c_to_h[{i},{j}]").X for i in suppliers.index for j in hubs.index]
yplant = [m.getVarByName(f"yplant[{i}]").X for i in plants.index]
yhub = [m.getVarByName(f"yhub[{i}]").X for i in hubs.index]
ytrain = [m.getVarByName(f"ytrain[{i},{j}]").X for i in hubs.index for j in plants.index]
ytruck = [m.getVarByName(f"ytruck[{i},{j}]").X for i in suppliers.index for j in hubs.index]
x_out = m.getVarByName("outside_supply").X

In [30]:
cost_per_litre = m.getObjective().getValue()/(x_out+(sum(hub_to_plant)*232))
print(cost_per_litre*1000)

1.3334860684552317


In [31]:
sum(ytruck)

8382.0

In [32]:
sum(county_to_hub)

2621788.017241381

In [33]:
sum(hub_to_plant)

2621788.0172415716

In [36]:
suppliers['supply'].sum() - sum(hub_to_plant)

431589.6910210545

In [37]:
np.nonzero(hub_to_plant)

(array([ 525,  692, 1323, 1516, 2002, 2351, 2696, 2837, 3327, 3687, 3828,
        4841, 5344], dtype=int64),)

In [38]:
np.nonzero(county_to_hub)

(array([   9,  127,  154,  187,  226,  247,  300,  334,  385,  424,  433,
         465,  498,  535,  580,  608,  643,  664,  748,  787,  811,  829,
         861,  902,  935,  964, 1037, 1103, 1171, 1197, 1243, 1273, 1276,
        1327, 1360, 1400, 1441, 1468, 1488, 1537, 1558, 1598, 1621, 1672,
        1756, 1862, 1888, 1961, 1994, 2024, 2053, 2134, 2173, 2197, 2286,
        2362, 2380, 2423, 2453, 2482, 2530, 2563, 2590, 2621, 2649, 2755,
        2779, 2808, 2882, 2915, 2984, 3017, 3040, 3080, 3157, 3187, 3256,
        3316, 3342, 3347, 3406, 3435, 3507, 3573, 3649, 3677, 3705, 3736,
        3809, 3868, 3913, 3938, 4042, 4087, 4120, 4144, 4165, 4202, 4233,
        4260, 4318, 4330, 4359, 4392, 4429, 4516, 4543, 4568, 4623, 4681,
        4697, 4723, 4761, 4801, 4827, 4912, 4920, 5023, 5052, 5086, 5143,
        5152, 5157, 5195, 5221, 5250, 5291, 5382, 5449, 5481, 5497, 5518,
        5563, 5593, 5657, 5698, 5718, 5751, 5815, 5869, 5956, 5992, 6015,
        6058, 6154, 6251, 6383, 6463, 

In [39]:
# roads_copy = roads.copy()
# roads_copy['c_to_h'] = c_to_h.gppd.X.to_frame()
# roads_copy = roads_copy.groupby('county').sum()

In [40]:
# roads_copy['supply'] = suppliers['supply'].copy()

In [41]:
# vio_list = []
# for index, row in roads_copy.iterrows():
#     if row['c_to_h'] > row['supply']+0.0001:
#         vio_list.append(index)

In [42]:
# roads_copy.loc[vio_list]

In [43]:
sum(hub_to_plant)*232

608254820.0000446

In [44]:
sum(yhub)

10.0

In [45]:
yhub

[-0.0,
 0.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 0.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0]

In [46]:
sum(yplant)

4.000000000000293